Iterative Solvers
===

So far we have used direct solvers to solve the linear system of equations. Although a direct solver can profit from the sparse matrix, it's arithmetic complexity is sub-optimal. For large-scale problems iterative solvers are a must.

The conjugate gradient (cg) method is the standard method for symmetric and positive definite matrices. It's convergence rate depends on a preconditioner, what is a cheap approximative inverse to the matrix.

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo

We generate a 3D geometry and mesh using the OCC constructive solid geometry (CSG) modeler:

In [ ]:
from netgen.occ import *
cube = Box((0,0,0),(1,1,1))
cyl = Cylinder((0,0.5,0.5),X, r=0.2, h=1)
cube.faces.name = "outer"
cyl.faces.name = "cyl"
shape = cube-cyl
DrawGeo(shape);

In [ ]:
ngmesh = OCCGeometry(shape).GenerateMesh(maxh=0.1)
for l in range(0):
    ngmesh.Refine()
mesh = Mesh(ngmesh)
mesh.Curve(3)
Draw (mesh);

In [ ]:
fes = H1(mesh, order=3, dirichlet="outer", wb_withoutedges=True)
print ("we have", fes.ndof, "unknowns")
u = fes.TrialFunction()
v = fes.TestFunction()

a = BilinearForm(fes)
a += grad(u)*grad(v)*dx

f = LinearForm(fes)
f += v*dx

# c = Preconditioner(a, "direct", inverse="sparsecholesky")
c = Preconditioner(a, "local")
# c = Preconditioner(a, "bddc")

gfu = GridFunction(fes)

assemble system and setup preconditioner in parallel:

In [ ]:
ngsglobals.msg_level=5
with TaskManager():
    a.Assemble()
    f.Assemble()

solve the system using the preconditioned conjugate gradient method:

In [ ]:
from ngsolve.krylovspace import CGSolver

with TaskManager():
    inv = CGSolver(mat=a.mat, pre=c.mat, printrates='\r', maxiter=400)
    gfu.vec.data = inv * f.vec

In [ ]:
Draw (gfu);